In [1]:
import pandas as pd
import numpy as np
from pandas.io.sql import read_sql
import os
import re
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 

In [2]:
# download the imdb dataset
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

--2019-10-18 23:21:14--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.2’

aclImdb_v1.tar.gz.2   7%[>                   ]   5.98M  2.30MB/s               ^C


In [ ]:
# in case we have not created the folder to save the data we create it using the magic command line
!mkdir datasets

In [ ]:
!tar -xzf aclImdb_v1.tar.gz --directory datasets/

In [3]:
# define the directory paths where we saved the imdb reviews
train_neg = 'datasets/aclImdb/train/neg'
train_pos = 'datasets/aclImdb/train/pos'

test_neg = 'datasets/aclImdb/test/neg'
test_pos = 'datasets/aclImdb/test/pos'

unsupervised = 'datasets/aclImdb/train/unsup'

In [4]:
# We now load the reviews into a dataframe and shuffle them 
# If you want to have a look at what for another implementation see https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb
from sklearn.utils import shuffle

def load_directory_data(directory):
    data = {}
    data["review"] = []

    for file_path in os.listdir(directory):
        with open(os.path.join(directory, file_path), "r") as f:
            data["review"].append(f.read())
  
    data = pd.DataFrame.from_dict(data)
  
    return data

In [5]:
# laod training and test data using a shuffle function
training_data = pd.concat([load_directory_data(train_neg), load_directory_data(train_pos)])
training_data = shuffle(training_data)
test_data = pd.concat([load_directory_data(test_neg), load_directory_data(test_pos)])
test_data = shuffle(test_data)
print(f'Lenght of training data is {len(training_data):,} and test data is {len(test_data):,}')

Lenght of training data is 25,000 and test data is 25,000


In [6]:
# laod training and test data using a shuffle function
unsupervised = pd.concat([load_directory_data(unsupervised)])
print(f'Lenght of training data is {len(unsupervised)}')

Lenght of training data is 50000


In [7]:
# split train and test
unsupervised = unsupervised.copy()
unsupervised = np.array_split(unsupervised, 2)
unsupervised_train = unsupervised[0]
unsupervised_test = unsupervised[1]
print(f'Lenght of training data is {len(unsupervised_train):,} and test data is {len(unsupervised_test):,}')

Lenght of training data is 25,000 and test data is 25,000


In [ ]:
!python -m spacy download en_core_web_lg

In [8]:
# The pretraining of BERT is composed of two different tasks next sentence prediction and masked language
# vocabulary prediction. We need to split our text into sentences. We will use SpaCy to perform that task.
# we filter sentences with less than 4 words. After spliting the text into sentences we will return a tuple
# composed of two sentences and a label detailing if the second sentence is the next sentence or if its a 
# random sentence. 
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

def sentence_tokenizer(text_sequence):
    ''' text_sequence: A text document to be splitted into sentences'''
    list_sentences= list(nlp.pipe(text_sequence, disable= ['tagger']))
    
    # create a list of sentences and make sure they are strings, else SpaCy throws and error
    list_sentences = [list(x.sents) for x in list_sentences]
    list_sentences = [[str(y) for y in x] for x in list_sentences]
  
    # filter if the sentence has less than 4 words
    list_sentences =  [[y for y in x if len(y)>=4] for x in list_sentences]
    
    # we package all the sentences in lists of two using iter and zip two create a list of lists
    list_sentences = [list(zip(*[iter(x)]*2)) for x in list_sentences]
    list_sentences = [item for sublist in list_sentences for item in sublist]
    
    # conver the list into a dataframe so we can split it in half.
    data_original = pd.DataFrame(list_sentences, columns=['sentence_1','sentence_2'])
    # add a label to indicate that the second sentence corresponds to the next sentence.
    data_original['label'] = 0
    
    # split the data into two. The second dataframe will have the next sentence shifted by 2 to guarantee that the
    # next sentence in that dataframe is not the next sentence. We use the np.roll function to shift an array a 
    # certain number of index positions. The original documentation of Hugginface identifies not next label as 1
    # so we also add that into the second dataframe.
    data = data_original.copy()
    data = np.array_split(data, 2)
    data_1= data[1]
    data_2 = data[0]
    data_2['sentence_2'] = np.roll(data_2['sentence_2'],2)
    data_2['label'] = 1
    
    #finally we concatenate the dataframe and return a list of tupples.
    data = pd.concat([data_1,data_2])
    data = list(data.itertuples(index=False, name=None))
    
    return data

In [9]:
##a = sentence_tokenizer('The water was good. But all the land was dry. a;. The weather channel is on Tv. I like that show a bit. My mom makes great brownies. Atilla terrorized soldiers in Athens. Gargarin lived here in a house')
sentence_test=['The water was good but all the land was dry. a;. The weather channel is on Tv. I like that show a bit. Yes that makes sense a lot. I love you honey. He was not correct. But sure as hell i was',
               'My mom makes great brownies. Atilla terrorized soldiers in Athens. Gargarin lived here in a house. I am a fan of the doges',
               'the navy came to my house. Here they are how is this possible']
a = sentence_tokenizer(sentence_test)
a

[('My mom makes great brownies.', 'Atilla terrorized soldiers in Athens.', 0),
 ('Gargarin lived here in a house.', 'I am a fan of the doges', 0),
 ('the navy came to my house.', 'Here they are how is this possible', 0),
 ('The water was good but all the land was dry.',
  'Yes that makes sense a lot.',
  1),
 ('I like that show a bit.', 'He was not correct.', 1),
 ('I love you honey.', 'The weather channel is on Tv.', 1)]

In [ ]:
# Explanation of the types of inputs that BERTforPretraining requires.



In [10]:
# we need to define a function to go over the tokens returned by the BERT tokenizer. We will skip special 
# characters and follow the proportion of masked tokens,
# 
# refer this file for the mask_token function https://github.com/google-research/bert/blob/master/create_pretraining_data.py

def mask_tokens(inputs):
    '''inputs: This the pair of sentences tokenized by the BERT tokenizer. This script returns 
    a the list of inputs and the masked labels. '''
    
    #copy the inputs
    labels = inputs.copy()
    
    # create a list of the indexes we are going to modify with a masked label
    index_to_replace = []
    # create a lost with the labels we are going to predict
    labels_filtered = []

    # create index of observations to replace, we skip masking the CLS and SEP special tokens
    for (index, element) in enumerate(labels):
        if element=='[CLS]'or element== '[SEP]':
            continue
        else:
            index_to_replace.append(index)
            labels_filtered.append(element)
        
    # get a list of the indices we are masking (85 percent mask, 10 percent random, 10 percent no change))
    # masked_indices= np.random.binomial(size=len(index_to_replace), n=2, p= 0.15)
    masked_indices = np.random.choice([0, 1], size=len(index_to_replace), p=[.85, .15])
    
    # create a list to store the labels of the masked tokens
    labels_masked_token = []
    # the masked indices contains the indices that we are going to mask, we iterate to get the index position 
    # of the indices we need to replace
    for index_position, index_bool in zip(index_to_replace, list(masked_indices)):
        if index_bool==1:
            labels_masked_token.append(index_position)
        else:
            continue 
    
    # once we have that letter we will replace those tokens with the masked token
    indices_replaced_mask_tokens = np.random.binomial(size=len(labels_masked_token), n=1, p= 0.8)
    
    # for the remaining 20 percent tokens half of the time we will replace them with a random token and the other
    # half we leave the original token in place.
    indices_replaced_mask_rand = np.random.binomial(size=len(labels_masked_token), n=1, p= 0.5) & ~indices_replaced_mask_tokens 
    
    # We now use our copy of the tokends, convert that into the ids of the vocabulary and start the replacement
    # process
    labels=tokenizer.convert_tokens_to_ids(labels)
    
    # We first replace the tokens masked with the id of the special [MASK] token in our vocabulary which 
    # corresponds to 100
    for boolean, true_index in zip(indices_replaced_mask_tokens, labels_masked_token):
        if boolean==1:
            labels[true_index]=100
    
    # For the tokens that need to be replaced with a random token in the range of the len of the vocabulary
    for boolean, true_index in zip(indices_replaced_mask_rand, labels_masked_token):
        if boolean==1:
            labels[true_index]= np.random.randint(0,len(tokenizer.vocab), size=1)[0]
    
    # create a list of the position masked tokens
    position_masked_tokens= []
    
    # we finally parse everything and add -1 to the tokens we did not mask.
    for index,token in enumerate(labels):
        if index in labels_masked_token:
            position_masked_tokens.append(tokenizer.convert_tokens_to_ids(inputs[index]))
        else:
            position_masked_tokens.append(-1)
    
    return labels, position_masked_tokens

In [11]:
# create class to load a review, split into sentences, check if the sentences are in line with max lenght
# reference this post https://github.com/ceshine/pytorch-pretrained-BERT/blob/master/notebooks/Next%20Sentence%20Prediction.ipynb?source=post_page-----1dbfe6a66f1d----------------------

import torch
from transformers import BertTokenizer, BertModel, BertForNextSentencePrediction

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_seq_length = 128

class Data_Processing(object):
    def __init__(self, text_sequence):
    
        if isinstance(text_sequence,list):
            pass
        else:
            text_sequence= list(text_sequence)
    
    # define the text column from the dataframe
        assert isinstance(text_sequence, list), 'Argument of wrong type!'


    
        self.text_column = sentence_tokenizer(text_sequence)

    def __len__(self):
        return len(self.text_column)  
    
# iter method to get each element at the time and tokenize it using bert        
  
    def __getitem__(self, index):
        #print(self.text_column[index][0])

        sentence_1_tokenized_text = tokenizer.tokenize(self.text_column[index][0])
        sentence_2_tokenized_text = tokenizer.tokenize(self.text_column[index][1])
    
        while True:
            total_length = len(sentence_1_tokenized_text) + len(sentence_2_tokenized_text)
            if total_length <= (max_seq_length-3):
                break
            if len(sentence_1_tokenized_text) > len(sentence_2_tokenized_text):
                sentence_1_tokenized_text.pop()
            else:
                sentence_2_tokenized_text.pop()
    
    #sentence_1_tokenized_tex_ids = tokenizer.convert_tokens_to_ids(sentence_1_tokenized_tex_ids)
    #sentence_2_tokenized_tex_ids = tokenizer.convert_tokens_to_ids(sentence_2_tokenized_tex_ids)

        inputs = ["[CLS]"] + sentence_1_tokenized_text + ["[SEP]"]
        inputs +=  sentence_2_tokenized_text + ["[SEP]"]
    
    #print(inputs)
    
        segment_ids = [0] * (len(sentence_1_tokenized_text)+2)
        segment_ids += [1] * (len(sentence_2_tokenized_text) + 1)

    
    # convert the inputs to ids (dict looking)
    
    #inputs_test,labels_mask = mask_tokens(inputs)
        input_ids, masked_lm_labels = mask_tokens(inputs)

    #input_ids = tokenizer.convert_tokens_to_ids(inputs)
    
     # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        padding_masked = [-1] * (max_seq_length - len(input_ids))


        #add padding in case it is necessary
        input_ids += padding
        segment_ids += padding
        attention_masks = [1 if x>0 else 0 for x in input_ids]
        masked_lm_labels += padding_masked

        #input_mask += padding
        assert len(input_ids) == max_seq_length
        assert inputs[0] == "[CLS]"
        assert input_ids[0]== 101
        assert len(attention_masks) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(masked_lm_labels) == max_seq_length
       
    #print(ids_review)
    
        input_ids = torch.tensor(input_ids)
        segment_ids = torch.tensor(segment_ids)
        attention_masks = torch.tensor(attention_masks)
        masked_lm_labels = torch.tensor(masked_lm_labels)
        #a = mask_tokens(inputs)

        sequence_label = self.text_column[index][2]



        return input_ids, segment_ids, attention_masks, masked_lm_labels, sequence_label
  

In [12]:
# reference for dataloaders https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
batch_size = 16

# create a class to process the traininga and test data
training_data = Data_Processing(unsupervised_train['review'])

test_data =  Data_Processing(unsupervised_test['review'])

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=10),
                   'val': DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=10)
                   }

dataset_sizes = {'train':len(training_data),
                'val':len(test_data)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [13]:
from transformers import BertConfig
config = BertConfig()
from transformers import BertForPreTraining
model = BertForPreTraining.from_pretrained(
    "bert-base-uncased",config=config)

In [14]:
import torch.optim as optim 
from torch.optim import lr_scheduler, AdamW
from transformers import WarmupLinearSchedule

lrlast = .001
#lrmain = .00001
lrmain = 2e-5


optim1 = optim.AdamW(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1) 

In [16]:
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score
import torch.nn.functional as F
number_steps= 8
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def train_model(model, optimizer, scheduler, num_epochs=3):
    best_eval_acc = 0.0
    for epoch in range(num_epochs):
    #zero the model gradients
        model.zero_grad()
        print(f'starting epoch {epoch+1} out of {num_epochs}')
        training_loss = []
        training_accuracy_next_sentence = []
        training_accuracy_vocab = []
        val_loss = []
        val_accuracy_next_sentence = []
        val_accuracy_vocab = []

        epoch_loss = 0.0

        # track number of batches and number of iterations
        counter = 0
        iterations = 0
        counter_val = 0
        iterations_val = 0

        # Iterate over data, feeding inputs, attention masks and labels
        model.train()
        for i, (inputs, segment_ids, attention_masks, masked_lm_labels, sequence_label) in enumerate(dataloaders_dict['train']):
            # add a counter that will register how many examples we have fed to thee model
            counter += batch_size
            iterations += 1
            # move the sequences, labels and masks to the GPU
            inputs = inputs.to(device) 
              #print(inputs)
            segment_ids = segment_ids.to(device)
            #print(segment_ids)
            attention_masks = attention_masks.to(device)
            #print(attention_mask)

            masked_lm_labels = masked_lm_labels.to(device)
            sequence_label = sequence_label.to(device)

            # feed the sequences to the model, specifying the attention mask
            outputs = model(inputs, token_type_ids = segment_ids, attention_mask= attention_masks, masked_lm_labels=masked_lm_labels,
                         next_sentence_label=sequence_label)

            # feed the logits returned by the model to the softmax to classify the function
            prediction_scores = outputs[1]
            seq_relationship_scores = outputs[2]

            # TO DO Accuracy of model

            #
            loss_reg = outputs[0]/number_steps

            #add the loss to the epoch loss
            epoch_loss += loss_reg
            training_loss.append(loss_reg)

            outputs[0].backward()

            # accumulate gradients and update every 4 batches
            if (i+1) % number_steps == 0:
                optimizer.step()                            # Now we can do an optimizer step
                model.zero_grad()

          # only present the information 
            if counter% 10 == 0:
            # get the predictions and the true labels out of the GPU
                preds_next_sentence = torch.argmax(seq_relationship_scores,dim=1).cpu().data.numpy()
                true_next_sentence = np.array(sequence_label.squeeze(0).cpu().data.numpy())

            #print('here', true_next_sentence.shape, preds_next_sentence.shape)

                accuracy_next_sentence = accuracy_score(true_next_sentence,preds_next_sentence)
                training_accuracy_next_sentence.append(accuracy_next_sentence)


            # repeat with softmax for vocabulary
            #masked_predictions = F.softmax(prediction_scores,dim=1)
            #masked_predictions = torch.argmax(masked_predictions, 1).cpu().data.numpy()

            #true_vocab = np.array(masked_lm_labels.cpu().data.numpy())
            #print('now here', masked_predictions.shape, true_vocab.shape)
            #print(masked_predictions)
            #accuracy_vocab = accuracy_score(masked_predictions,true_vocab)
            #accuracy_vocab = sum(masked_predictions == true_vocab.squeeze(1))/len(true_vocab)
                print(f'current training loss is {np.sum(training_loss)/iterations} and next sent accuracy is {np.mean(training_accuracy_next_sentence):,.2%}')


        with torch.no_grad():
            model.eval()
            counter_val = 0
            iterations_val = 0
            for i, (inputs, segment_ids, attention_masks, masked_lm_labels, sequence_label) in enumerate(dataloaders_dict['val']):
                counter_val += batch_size
                iterations_val += 1

            inputs = inputs.to(device) 
            #print(inputs)
            segment_ids = segment_ids.to(device)
            #print(segment_ids)
            attention_masks = attention_masks.to(device)
            #print(attention_mask)

            masked_lm_labels = masked_lm_labels.to(device)
            sequence_label = sequence_label.to(device)

            # feed the sequences to the model, specifying the attention mask
            outputs = model(inputs, token_type_ids = segment_ids, attention_mask= attention_masks, masked_lm_labels=masked_lm_labels,
                          next_sentence_label=sequence_label)

            prediction_scores = outputs[1]
            seq_relationship_scores = outputs[2]

            # calculate the loss function, squeeze the labels so their shapes are compatible
            loss_eval = outputs[0] 
            val_loss.append(loss_eval)

            if counter_val % 10 == 0:
                # get the predictions and the true labels out of the GPU for validation
                preds_next_sentence = torch.argmax(seq_relationship_scores,dim=1).cpu().data.numpy()
                true_next_sentence = np.array(sequence_label.squeeze(0).cpu().data.numpy())
                accuracy_next_sentence = accuracy_score(true_next_sentence,preds_next_sentence)

             
                val_accuracy_next_sentence.append(accuracy_next_sentence)
              #val_accuracy_vocab.append(accuracy_vocab)
                print(f'current val loss is {np.sum(val_loss)/iterations_val} and next sent accuracy val is {np.mean(val_accuracy_next_sentence):,.2%}')
              # and vocab accuracy is \
             #{np.mean(val_accuracy_vocab):,.2%}')

    print(f'For epoch {epoch+1} training loss is {np.sum(training_loss)/iterations}, \
    training accuracy is {np.mean(training_accuracy_next_sentence):,.2%}, Validation loss is {np.sum(val_loss)/iterations_val} \
    and validation accuracy is {np.mean(val_accuracy_next_sentence):,.2%}')

    eval_acc = np.mean(val_accuracy_next_sentence)
    if eval_acc >= best_eval_acc:
        best_eval_acc = eval_acc
        print(f'saving the model with validation accuracy of {eval_acc:,.2%} ')
        torch.save(model.state_dict(), 'bert_imdb_pretrain.pth')
        torch.save(optimizer_ft.state_dict(), 'bert_imdb_optimiser_pretrain.pth')

            # model loading weights

        model.load_state_dict(torch.load('bert_imdb_pretrain.pth'))
        optimizer.load_state_dict(torch.load('bert_imdb_optimiser_pretrain.pth'))

    else:
        print(f'model did not improve')

      # load     
          #if epoch_loss<previous_loss:
          #  print(f'saving the model with epoch loss {epoch_loss} of and accuracy of {np.mean(accuracy_):,.2%}')
          #  torch.save(model.state_dict(), 'bert_imdb.pth')
          #  torch.save(optimiser.state_dict(), 'bert_imdb_optimiser.pth')
    return model

cuda:0


In [17]:
model.to(device)
model_ft1 = train_model(model, optimizer_ft, exp_lr_scheduler,
                       num_epochs=3)

starting epoch 1 out of 3
current training loss is 1.6639378070831299 and next sent accuracy is 62.50%
current training loss is 1.581919550895691 and next sent accuracy is 62.50%
current training loss is 1.5170029401779175 and next sent accuracy is 64.58%
current training loss is 1.4409641027450562 and next sent accuracy is 62.50%
current training loss is 1.3855009078979492 and next sent accuracy is 60.00%
current training loss is 1.3443255424499512 and next sent accuracy is 55.21%
current training loss is 1.2949395179748535 and next sent accuracy is 55.36%
current training loss is 1.2569431066513062 and next sent accuracy is 53.91%
current training loss is 1.2042973041534424 and next sent accuracy is 54.86%
current training loss is 1.1623518466949463 and next sent accuracy is 56.25%
current training loss is 1.12083899974823 and next sent accuracy is 57.39%
current training loss is 1.0812482833862305 and next sent accuracy is 57.29%
current training loss is 1.0500850677490234 and next 

current training loss is 0.5113918781280518 and next sent accuracy is 62.85%
current training loss is 0.5101706981658936 and next sent accuracy is 62.90%
current training loss is 0.5093107223510742 and next sent accuracy is 62.90%
current training loss is 0.5083179473876953 and next sent accuracy is 62.95%
current training loss is 0.5070686936378479 and next sent accuracy is 63.06%
current training loss is 0.5057573318481445 and next sent accuracy is 63.11%
current training loss is 0.5047687888145447 and next sent accuracy is 63.21%
current training loss is 0.5040748715400696 and next sent accuracy is 63.15%
current training loss is 0.5033966898918152 and next sent accuracy is 63.31%
current training loss is 0.5023567080497742 and next sent accuracy is 63.25%
current training loss is 0.5014007687568665 and next sent accuracy is 63.24%
current training loss is 0.5005921721458435 and next sent accuracy is 63.29%
current training loss is 0.49965181946754456 and next sent accuracy is 63.12

current training loss is 0.44864511489868164 and next sent accuracy is 64.37%
current training loss is 0.44827452301979065 and next sent accuracy is 64.45%
current training loss is 0.4477362632751465 and next sent accuracy is 64.47%
current training loss is 0.4474506676197052 and next sent accuracy is 64.43%
current training loss is 0.4469054639339447 and next sent accuracy is 64.51%
current training loss is 0.44660550355911255 and next sent accuracy is 64.47%
current training loss is 0.446384072303772 and next sent accuracy is 64.43%
current training loss is 0.4459788203239441 and next sent accuracy is 64.54%
current training loss is 0.445840448141098 and next sent accuracy is 64.56%
current training loss is 0.44548746943473816 and next sent accuracy is 64.55%
current training loss is 0.4453352689743042 and next sent accuracy is 64.56%
current training loss is 0.44508811831474304 and next sent accuracy is 64.53%
current training loss is 0.44466501474380493 and next sent accuracy is 64

current training loss is 0.4235115647315979 and next sent accuracy is 65.49%
current training loss is 0.4232337176799774 and next sent accuracy is 65.52%
current training loss is 0.423064261674881 and next sent accuracy is 65.55%
current training loss is 0.422895222902298 and next sent accuracy is 65.50%
current training loss is 0.4227847754955292 and next sent accuracy is 65.45%
current training loss is 0.4225139021873474 and next sent accuracy is 65.42%
current training loss is 0.4223492741584778 and next sent accuracy is 65.43%
current training loss is 0.42219144105911255 and next sent accuracy is 65.41%
current training loss is 0.42203909158706665 and next sent accuracy is 65.47%
current training loss is 0.42191383242607117 and next sent accuracy is 65.48%
current training loss is 0.421673059463501 and next sent accuracy is 65.44%
current training loss is 0.42145127058029175 and next sent accuracy is 65.43%
current training loss is 0.42121022939682007 and next sent accuracy is 65.4

current training loss is 0.40930309891700745 and next sent accuracy is 66.11%
current training loss is 0.409106969833374 and next sent accuracy is 66.13%
current training loss is 0.4090023338794708 and next sent accuracy is 66.18%
current training loss is 0.40889227390289307 and next sent accuracy is 66.19%
current training loss is 0.40873855352401733 and next sent accuracy is 66.21%
current training loss is 0.40863436460494995 and next sent accuracy is 66.26%
current training loss is 0.40843167901039124 and next sent accuracy is 66.25%
current training loss is 0.40827107429504395 and next sent accuracy is 66.28%
current training loss is 0.40822818875312805 and next sent accuracy is 66.26%
current training loss is 0.4081786870956421 and next sent accuracy is 66.29%
current training loss is 0.40806838870048523 and next sent accuracy is 66.33%
current training loss is 0.4080184996128082 and next sent accuracy is 66.33%
current training loss is 0.4078954756259918 and next sent accuracy is

...

current training loss is 0.3180816173553467 and next sent accuracy is 77.90%
current training loss is 0.3180885910987854 and next sent accuracy is 77.89%
current training loss is 0.3180789053440094 and next sent accuracy is 77.90%
current training loss is 0.3180878460407257 and next sent accuracy is 77.89%
current training loss is 0.3181034326553345 and next sent accuracy is 77.89%
current training loss is 0.31809359788894653 and next sent accuracy is 77.89%
current training loss is 0.31808504462242126 and next sent accuracy is 77.89%
current training loss is 0.3180781304836273 and next sent accuracy is 77.88%
current training loss is 0.31809040904045105 and next sent accuracy is 77.89%
current training loss is 0.31809908151626587 and next sent accuracy is 77.88%
current training loss is 0.31810155510902405 and next sent accuracy is 77.88%
current training loss is 0.31808313727378845 and next sent accuracy is 77.89%
current training loss is 0.31809696555137634 and next sent accuracy is

current training loss is 0.3179640471935272 and next sent accuracy is 77.94%
current training loss is 0.31795862317085266 and next sent accuracy is 77.94%
current training loss is 0.31793710589408875 and next sent accuracy is 77.94%
current training loss is 0.31793543696403503 and next sent accuracy is 77.94%
current training loss is 0.31792908906936646 and next sent accuracy is 77.94%
current training loss is 0.31792429089546204 and next sent accuracy is 77.94%
current training loss is 0.317922443151474 and next sent accuracy is 77.94%
current training loss is 0.3179325759410858 and next sent accuracy is 77.94%
current training loss is 0.31793642044067383 and next sent accuracy is 77.94%
current training loss is 0.3179285228252411 and next sent accuracy is 77.94%
current training loss is 0.31792640686035156 and next sent accuracy is 77.94%
current training loss is 0.31790128350257874 and next sent accuracy is 77.94%
current training loss is 0.3178919553756714 and next sent accuracy is

current training loss is 0.3179088234901428 and next sent accuracy is 77.94%
current training loss is 0.317912220954895 and next sent accuracy is 77.94%
current training loss is 0.31791266798973083 and next sent accuracy is 77.95%
current training loss is 0.3179221451282501 and next sent accuracy is 77.94%
current training loss is 0.3179285526275635 and next sent accuracy is 77.94%
current training loss is 0.31792572140693665 and next sent accuracy is 77.94%
current training loss is 0.3179197311401367 and next sent accuracy is 77.95%
current training loss is 0.3179172873497009 and next sent accuracy is 77.95%
current training loss is 0.3179195523262024 and next sent accuracy is 77.95%
current training loss is 0.31791582703590393 and next sent accuracy is 77.95%
current training loss is 0.3179070055484772 and next sent accuracy is 77.95%
current training loss is 0.31790053844451904 and next sent accuracy is 77.95%
current training loss is 0.3179052472114563 and next sent accuracy is 77.

current training loss is 0.31773102283477783 and next sent accuracy is 77.97%
current training loss is 0.31772324442863464 and next sent accuracy is 77.98%
current training loss is 0.317739874124527 and next sent accuracy is 77.97%
current training loss is 0.31773945689201355 and next sent accuracy is 77.98%
current training loss is 0.31773924827575684 and next sent accuracy is 77.97%
current training loss is 0.3177329897880554 and next sent accuracy is 77.97%
current training loss is 0.3177400827407837 and next sent accuracy is 77.97%
current training loss is 0.31773701310157776 and next sent accuracy is 77.97%
current training loss is 0.31773123145103455 and next sent accuracy is 77.97%
current training loss is 0.31773674488067627 and next sent accuracy is 77.97%
current training loss is 0.31772175431251526 and next sent accuracy is 77.97%
current training loss is 0.31772249937057495 and next sent accuracy is 77.97%
current training loss is 0.317720890045166 and next sent accuracy is

/home/jlealtru/anaconda3/envs/pytorch/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jlealtru/anaconda3/envs/pytorch/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [18]:
import requests
resp = requests.post('https://textbelt.com/text', {
  'phone': '4123131113',
  'message': 'Work is done',
  'key': 'textbelt']
})
print(resp.json())

{'success': True, 'textId': '27121571500387835', 'quotaRemaining': 0}
